# Análise de sentimentos de tweets brasileiros nos períodos anterior e inicial à pandemia de Covid-19

## Extração de dados brasileiros e armazenamento no banco de dados

***
### Base GeoCoV19

O artigo GeoCov19 disponibiliza uma base de dados contendo mais de **524 milhões de registros de tweets**, em **62 línguas**, que fazem referência à Covid-19. Deste total, **94% do conjunto de dados estão geolocalizados**.

Para a inclusão de informações de geolocalização foram consideradas informações de localização **providas pelo próprio Twitter (com diferentes níveis de confiabilidade) e localizações extraídas a partir dos textos dos tweets (topônimos)**. A extração a partir dos textos considera qualquer menção a algum local como uma informação de localização válida.

Em atendimento à políticas do Twitter, **os dados disponibilizados não possuem os textos dos tweets**. O autor disponibiliza uma **ferramenta para obtenção do conteúdo integral dos tweets** através de seus "ids", procedimento conhecido como *hydrate*.

Os dados coletados tem por objetivo capacitar comunidades de pesquisa a avaliar como as sociedades estão lidando coletivamente com a crise do Covid-19, desenvolver métodos para identificar notícias falsas, entender lacunas de conhecimento, contruir modelos de previsão, desenvolver alertas à doenças, entre outros.

### Objetivo

Extração, seleção e análise de sentimentos de **tweets brasileiros** durante o período de **01/02/2020 a 01/05/2020**.

### Informações Técnicas

#### Estrutura do arquivo JSON contendo tweets e informações de localização

1. **tweet_id**: it represents the Twitter provided id of a tweet

2. **created_at**: it represents the Twitter provided "created_at" date and time in UTC 

3. **user_id**: it represents the Twitter provided user id

4. **geo_source**: this field shows one of the four values: (i) coordinates, (ii) place, (iii) user_location, or (iv) tweet_text. The value depends on the availability of these fields. The remaining keys can have the following location_json inside them: {"country_code":"us","state":"California","county":"San Francisco","city":"San Francisco"}.

5. **user_location**: It can have a "location_json" as described above or an empty JSON {}. This field uses the "location" profile meta-data of a Twitter user and represents the user declared location in the text format. We resolve the text to a location. 

6. **geo**: represents the "geo" field provided by Twitter. We resolve the provided latitude and longitude values to locations. It can have a "location_json" as described above or an empty JSON {}.

7. **tweet_locations**: This field can have an array of "location_json" as described above [location_json1, location_json2] or an empty array []. This field uses the tweet content to find toponyms. 

8. **place**: It can have a "location_json" described above or an empty JSON {}. It represents the Twitter-provided "place" field.

### Solução Técnica

Foi implementada uma solução para a **leitura e seleção de tweets, a partir dos arquivos de dados providos pelo  GeoCoV19, selecionando registros que faziam referência ao Brasil**. A partir dos registros selecionados foi desenvolvida uma solução para análise de resultados obtidos a partir dos sentimentos das cidades com **scores** mais negativos e mais positivos.

Os dados foram disponibilizados em arquivos diários compactados no formato "zip", totalizando **90 arquivos**. A solução realiza a descompactação de cada um desses arquivos (que possuem formato JSON) selecionando os registros desejados e criando um novo arquivo JSON com esses registros. 

Os arquivos disponibilizados possuem informações de **geolocalização dos tweets** (citadas no item anterior) e também um **atributo informando quais destas localizações fornecidas é a mais precisa** (atributo "geo_source"). Através deste atributo selecionamos os registros desejados (contendo a coluna country_code = "br") durante a leitura dos arquivos JSON.

Os registros selecionados foram armazenados em um banco de dados **MongoDB**. Em um segundo momento, foram realizados os *hydrates* desses tweets, a partir de seus *ids*, utilizando a ferramenta **Twarc**. Os textos foram traduzidos com a utilização da ferramenta **Googletrans** para que pudessem ter seus sentimentos analisados, com a utilização do **Vader Sentiment**, ferramenta desenvolvida especialmente para a utilização em textos de redes sociais.

Por último, os resultados dos sentimentos obtidos foram analisados com a utilização de recursos das bibliotecas **Pandas e Matplotlib**, para geração de gráficos e **NTLK** e **Spacy**, para processamento de linguagem natural, entre outros.

***
### Importações gerais

In [1]:
import datetime
import pandas as pd

### Configurações iniciais

#### Variáveis utilizadas na extração de dados

In [2]:
# Diretório base
#home = '/home/mario/Arquivos'
home = '/media/mario/Dados2/Arquivos'

# Diretório dos arquivos zip 
zip_dir = home + '/input/geo'

# Diretório de descompactação dos arquivos zip (arquivos json)
json_dir = home + '/output/json'

# Diretório dos arquivos json com geolocalizações brasileiras processados a partir dos zips descompactados
geo_dir = home + '/output/geo'

# Diretório dos arquivos de ids
ids_dir = home + '/output/ids'

# Diretório de downloads realizados pelo Twarc
downloads_dir = home + '/downloads'

# Diretório de tweets com textos
tweets_dir = home + '/output/tweets'

#### Conexão ao banco de dados

In [3]:
# Criando estrutura do banco de dados
from pymongo import MongoClient

# Conexão com o servidor do MongoDB
client = MongoClient('localhost', 27017)

# Conexão com a base de dados do mongoDB
db = client.SpedDB

# Coleção onde serão inseridos os dados
collection = db.tweets_brasil_test2
#collection = db.tweets_brasil

***
### Realização da extração de dados brasileiros

#### Funções auxiliares para extração de dados

In [4]:
import ijson
import json
import os
import zipfile
import numpy as np
import pandas as pd

In [5]:
# Função para extração dos arquivos zip e realização da leitura dos arquivos JSON descompactados
def read_files(zip_dir, json_dir, geo_dir, ids_dir, country_code):
    
    print("Extraindo arquivos...")
    extract_files(zip_dir, json_dir)
    
    json_files = list_files(json_dir, ".json")
        
    total_arquivos = len(json_files)
    total_arquivos_processados = 0
    total_tweets_validos = 0
    
    print(str(total_arquivos) + " arquivo(s) extraídos(s)")
    print("Processando arquivo(s) extraído(s)...")
    
    # Percorrendo arquivos do diretório
    for file in json_files:
        
        total_arquivos_processados = total_arquivos_processados + 1
        
        # Lendo o arquivo JSON extraído     
        num_linhas = sum(1 for line in open(file))
        print("Lendo arquivo '"+get_filename(file)+" com "+str(num_linhas)+" linhas")
        new_tweets = read_tweets(file, country_code)
        print("Tweets válidos encontrados: "+str(len(new_tweets)))
        total_tweets_validos = total_tweets_validos + len(new_tweets)
           
        # Criando dataframe de geolocalização de tweets
        df_geo = create_df_tweets(new_tweets)   
            
        # Escrevendo json com com as geolocalizações brasileiras
        filename = get_filename(file)
        csv_path = geo_dir + os.path.sep + filename
        print("-> Gerando arquivo json '"+get_filename(csv_path))
        df_geo.to_json(csv_path, orient='records', force_ascii=False)
        
        # Escrevendo json com 
        output_file_path = ids_dir + os.path.sep + filename + '_ids.csv'
        print("-> Gerando arquivo com ids '"+output_file_path)
        df_geo.to_csv(output_file_path, sep=';',encoding='utf-8', index=False, header=False, columns=['tweet_id'])        
            
        df_geo = None
        print("Tweets válidos até o momento: "+str(total_tweets_validos))
                    

In [6]:
# Função para extração dos arquivos zip
def extract_files(zip_dir, json_dir):
    
    zips = list_files(zip_dir, ".zip") 
    
    total_arquivos_processados = 0
    total_arquivos = len(zips)
    
    for file in zips:
        
        total_arquivos_processados = total_arquivos_processados + 1
        
        if (is_file_extracted(file, json_dir)):
            print("-> Arquivo ''"+get_filename(file)+ "' já extraído anteriormente"+" ("+str(total_arquivos_processados)+"/"+str(total_arquivos)+")")
        else:
            # Extraindo arquivo zip
            zip = zipfile.ZipFile(file)
            print("-> Extraindo arquivo '"+get_filename(zip.filename)+"' ("+str(total_arquivos_processados)+"/"+str(total_arquivos)+")")
            zip.extractall(json_dir)
            zip.close()

In [7]:
# Função para a criação de um dataframe a partir dos tweets gerados com os atributos desejados
def create_df_tweets(tweets):
    
    tweet_columns = ['tweet_id','created_at','geo_source','state','city']
    df = pd.DataFrame(tweets, columns = tweet_columns)
    
    # Modificando os tipos de colunas para otimização de espaço
    df.tweet_id = df.tweet_id.astype('int64')
    df.state = df.state.astype('category')
    df.city = df.city.astype('category')
    
    # Informando valores nulos
    df.text = np.nan
    df.score = np.nan
    
    return df

In [8]:
# Função para retornar a localização do tweet a ser considerada (dentre as várias localizações que podem ter sido informadas)
def select_location(tweet, country_code):
    
    if tweet['geo_source'] == 'coordinates':
        return tweet['geo']
    if tweet['geo_source'] == 'place':
        return tweet['place']
    if tweet['geo_source'] == 'user_location':
        return tweet['user_location']
    if tweet['geo_source'] == 'tweet_text':
        for location in tweet['tweet_locations']:
            if location['country_code'] == country_code:
                return location
    else: 
        return {}

In [9]:
# Função para identificar se o tweet que está sendo lido pertence ao país desejado
def is_valid_tweet(tweet, country_code):
    
    # Verificando preliminarmente se os dados pertencem a outros países diferentes do Brasil
    if tweet['geo_source'] == 'geo' and 'country_code' in tweet['geo'] and tweet['geo']['country_code'] != country_code:
        return False
    if tweet['geo_source'] == 'place' and 'country_code' in tweet['place'] and tweet['place']['country_code'] != country_code:
        return False
    if tweet['geo_source'] == 'user_location' and 'country_code' in tweet['user_location'] and tweet['user_location']['country_code'] != country_code:
        return False
    
    # Verificando se as informações de cidades e estados são nulas
    if tweet['geo_source'] == 'geo' and 'country_code' in tweet['geo'] and tweet['geo']['country_code'] == country_code and 'state' in tweet['geo'] and tweet['geo']['state'] != np.nan and 'city' in tweet['geo'] and tweet['geo']['city'] != np.nan:
        return True
    if tweet['geo_source'] == 'place' and 'country_code' in tweet['place'] and tweet['place']['country_code'] == country_code and 'state' in tweet['place'] and tweet['place']['state'] != np.nan and 'city' in tweet['place'] and tweet['place']['city'] != np.nan:
        return True
    if tweet['geo_source'] == 'user_location' and 'country_code' in tweet['user_location'] and tweet['user_location']['country_code'] == country_code and 'state' in tweet['user_location'] and tweet['user_location']['state'] != np.nan  and 'city' in tweet['user_location'] and tweet['user_location']['city'] != np.nan:
        return True
    
    # Caso as informações de localização não estejam presentes nos atributos 'geo', 'place' e 'user_location'
    if tweet['geo_source'] == 'tweet_text':
        for location in tweet['tweet_locations']:
            if location['country_code'] == country_code:
                if 'state' in location and location['state'] != np.nan and 'city' in location and location['city'] != np.nan:
                    return True
                else:
                    return False

    return False  

In [10]:
# Função para a criação de um novo registro de tweet com atributos desejados dos tweets do arquivo original
def create_new_tweet(tweet, country_code):      
    new_tweet = {}    
    location = select_location(tweet, country_code)    
    new_tweet['tweet_id'] = tweet['tweet_id']
    new_tweet['created_at'] = tweet['created_at']
    new_tweet['geo_source'] = tweet['geo_source']
    new_tweet['country_code'] = (location['country_code'] if 'country_code' in location else None)
    new_tweet['state'] = (location['state'] if 'state' in location else None)
    new_tweet['city'] = (location['city'] if 'city' in location else None)   
    return new_tweet

In [11]:
# Função para realizar a leitura de tweets de um país desejado a partir do arquivo JSON
def read_tweets(file, country_code):
    
    with open(file, 'r') as f:
    
        # Array de tweets tratados
        new_tweets = []

        # Realizando leitura iterativa do arquivo (todas as colunas selecionadas)
        objects = ijson.items(f, "", multiple_values=True)

        # Selecionando os tweets desejados
        tweets = (item for item in objects if is_valid_tweet(item, country_code))

        for tweet in tweets:
            new_tweets.append(create_new_tweet(tweet, country_code))
            
    return new_tweets   

In [12]:
def is_file_extracted(file, dir):
    
    filename = get_filename(file)
    filename = filename.replace(".zip",".json")
    filename = dir + os.path.sep + filename
    return os.path.isfile(filename)

In [13]:
def list_files(dir, type):
    
    caminhos = [os.path.join(dir, nome) for nome in os.listdir(dir)]
    arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
    valid_files = [arq for arq in arquivos if arq.lower().endswith(type)]
                
    return sorted(valid_files)

In [14]:
def get_filename(file):
    
    split = file.split(os.path.sep)
    size = len(split)
    filename = split[size-1]
    return filename

#### Funções auxiliares para processamento e limpeza de textos

In [15]:
import nltk
import re

from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
# Função auxiliar para remoção de caracteres indesejados nos textos dos Tweets
chars=["\"","!","@","#","$","%","&","*","(",")","-","_","`","'","{","[","]","}","^","~",",",".",";",":","\","," "]

def clean_words(words):
    
    new_words = []    
    for word in words:
         # Verificando se o caracter pertence ao ASCII
        if(all(ord(char) < 128 for char in word)):
            for letter in word:
                if letter in chars:
                    word=word.replace(letter,"")
        new_words.append(word)            
    return new_words

In [18]:
# Função auxiliar para remover urls nos textos dos Tweets
def clean_urls(words):   
    
    new_words = []    
    for word in words:              
        if (bool(re.match('http', word)) == False) and (bool(re.match('//tco', word)) == False) and (bool(re.match('//t.co', word)) == False) and (bool(re.match('RT', word)) == False):
            new_words.append(word)
    return new_words

In [19]:
# Função para remover "stopwords" dos textos dos Tweets
def remove_stopwords(words, language): 
    
    stopwords = nltk.corpus.stopwords.words(language) + list(punctuation)    
    new_words=[]
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

In [20]:
# Realiza o processo de limpeza do texto
def process_tweet(text, language):
    words = text.split()
    words = clean_words(words)
    words = clean_urls(words)
    words = remove_stopwords(words, language)
    return words

#### Realização da leitura dos arquivos zip e extração dos dados desejados

In [21]:
# Realizando a extração dos arquivos zip
read_files(zip_dir, json_dir, geo_dir, ids_dir, 'br')

Extraindo arquivos...
-> Arquivo ''geo_2020-02-01.zip' já extraído anteriormente (1/2)
-> Arquivo ''geo_2020-02-02.zip' já extraído anteriormente (2/2)
2 arquivo(s) extraídos(s)
Processando arquivo(s) extraído(s)...
Lendo arquivo 'geo_2020-02-01.json com 666573 linhas
Tweets válidos encontrados: 954
-> Gerando arquivo json 'geo_2020-02-01.json
-> Gerando arquivo com ids '/media/mario/Dados2/Arquivos/output/ids/geo_2020-02-01.json_ids.csv
Tweets válidos até o momento: 954
Lendo arquivo 'geo_2020-02-02.json com 1462153 linhas
Tweets válidos encontrados: 10798
-> Gerando arquivo json 'geo_2020-02-02.json
-> Gerando arquivo com ids '/media/mario/Dados2/Arquivos/output/ids/geo_2020-02-02.json_ids.csv
Tweets válidos até o momento: 11752


***
### Armazenamento dos tweets brasileiros no banco de dados

#### Funções auxiliares para armazenamento de dados

In [22]:
# Função para inserção de tweets no banco de dados, a partir de arquivos JSON
def create_db(collection, json_files):
    
    for file in json_files: 
        print('Criando tweets do arquivo '+file)
        with open(file) as json_file: 
            tweets = json.load(json_file)  
            for tweet in tweets:
                date = datetime.datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y')   
                tweet['created_at'] = date
                tweet['period'] = str(date.year) + "_" + str(date.month).zfill(2)
                tweet['text'] = None
                tweet['lang'] = None
                tweet['score'] = None
                collection.insert_one(tweet)

In [23]:
# Função para criação de índice no banco de dados
def create_index(collection, column):
    
    collection.create_index(column)

#### Realização do armazenamento de tweets 

In [24]:
# Lendo arquivos json com registros brasileiros
json_files = list_files(geo_dir, ".json")

# Criando banco de dados de tweets com as geolocalizações brasileiras
create_db(collection, json_files)

Criando tweets do arquivo /media/mario/Dados2/Arquivos/output/geo/geo_2020-02-01.json
Criando tweets do arquivo /media/mario/Dados2/Arquivos/output/geo/geo_2020-02-02.json


In [25]:
# Criando index para outros atributos alvos de selects
create_index(collection, 'tweet_id')
create_index(collection, 'state')
create_index(collection, 'city')
create_index(collection, 'lang')
create_index(collection, 'geo_source')
create_index(collection, 'created_at')
create_index(collection, 'polarity')

In [26]:
# Retornando quantidade de tweets inseridos
collection.count_documents({})

11752

In [27]:
# Retornando o primeiro tweet como exemplo
collection.find_one({})

{'_id': ObjectId('5ffc340721bf9fca746904c1'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'period': '2020_02',
 'text': None,
 'lang': None,
 'score': None}

***
### Hydrate de Tweets

A partir dos arquivos de ids gerados no procedimento de leitura dos arquivos zips, foi realizado o *hydrate* dos tweets, a partir desses ids, utilizando a ferramento Twarc.

#### Funções auxiliares para atualizações no banco de dados

In [28]:
# Função para atualização de registro de um tweet no banco de dados
def update_tweets_db(collection, json_files): 
    
    for file in json_files:
        print('Atualizando tweets do arquivo '+file)
        for line in open(file, 'r'):
            tweet = json.loads(line)
            collection.update_one({"tweet_id": tweet['id']}, {'$set':{"text": tweet['full_text'], "lang": tweet['lang']}})

#### Atualização de textos de tweets no banco de dados

Realizando a atualização no banco de dados com os textos dos tweets retornados pelo *Twarc*

In [29]:
# Listando arquivos json gerados pelo Twarc
twarc_files = list_files(downloads_dir, '.json')

# Atualizando banco de dados com os atributos Text e Lang a partir dos arquivos Json gerados pelo Twarc
update_tweets_db(collection, twarc_files)

Atualizando tweets do arquivo /media/mario/Dados2/Arquivos/downloads/tweets_2020-02-01.json
Atualizando tweets do arquivo /media/mario/Dados2/Arquivos/downloads/tweets_2020-02-02.json


In [30]:
# Retornando quantidade de tweets com textos não nulos
collection.count_documents({'text': {'$ne':None}})

8494

In [31]:
# Retornando o primeiro tweet como exemplo
collection.find_one({})

{'_id': ObjectId('5ffc340721bf9fca746904c1'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'period': '2020_02',
 'text': 'Director General of Health Services Dr. Anil Jasinghe confirmed the Chinese woman admitted at IDH has recovered and can be discharged by tomorrow #Lka #SriLanka #coronavirus',
 'lang': 'en',
 'score': None}

### Fontes

(1) GeoCoV19: A Dataset of Hundreds of Millions ofMultilingual COVID-19 Tweets with Location Information<br>

(2) Paper Info, Statics and Downloads - https://crisisnlp.qcri.org/covid19<br>

(3) Pyhton Documentation - https://docs.python.org/3/ 

(4) Muhammad Imran, Prasenjit Mitra, Carlos Castillo: Twitter as a Lifeline: Human-annotated Twitter Corpora for NLP of Crisis-related Messages. In Proceedings of the 10th Language Resources and Evaluation Conference (LREC), pp. 1638-1643. May 2016, Portorož, Slovenia.

(5) How to Generate API Key, Consumer Token, Access Key for Twitter OAuth - https://themepacific.com/how-to-generate-api-key-consumer-token-access-key-for-twitter-oauth/994/

(6) Iterative JSON parser with a standard Python iterator interface - https://pypi.org/project/ijson/

(7) Make working with large DataFrames easier, at least for your memory - https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4

(8) Twitter API Documentation - Tweet Location Metadata - https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/geo-objects

(9) Twarc - https://github.com/DocNow/twarc

(10) Map Polygon - https://www.keene.edu/campus/maps/tool/

(11) MongoBD Documentation - https://docs.mongodb.com/manual/reference/command/count/

(12) Free Google Translator API for Pyhton - https://pypi.org/project/googletrans/